In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

In [2]:
test0 = ['.....',
         '.S-7.',
         '.|.|.',
         '.L-J.',
         '.....']

test1 = ['..F7.',
         '.FJ|.',
         'SJ.L7',
         '|F--J',
         'LJ...']

data = np.genfromtxt('day10_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
def get_graph(data):
    new_data = []
    for line in data:
        new_data.append(list(line))
        
    new_data = np.array(new_data)
    start = np.where(new_data == 'S')
    
    return new_data, (start[0][0], start[1][0])

#Check if position is traversable
def good_pos_part1(graph, pos, cur_val, dxdy):
    x = pos[0]
    y = pos[1]
    if x < 0 or x >= len(graph):
        return False
    if y < 0 or y >= len(graph[x]):
        return False
    
    mapping = {'|': {(-1,0): ['|', '7', 'F'], (1,0):  ['|', 'L', 'J']},
               '-': {(0,-1): ['-', 'L', 'F'], (0,1):  ['-', 'J', '7']},
               'L': {(-1,0): ['|', '7', 'F'], (0,1):  ['-', 'J', '7']},
               'J': {(-1,0): ['|', '7', 'F'], (0,-1): ['-', 'L', 'F']},
               '7': {(1,0):  ['|', 'L', 'J'], (0,-1): ['-', 'L', 'F']},
               'F': {(1,0):  ['|', 'L', 'J'], (0,1):  ['-', 'J', '7']},
               'S': {(-1,0): ['|', '7', 'F'], (1,0):  ['|', 'L', 'J'],
                     (0,-1):  ['-', 'L', 'F'], (0,1): ['-', 'J', '7']}}
    if graph[x][y] not in mapping.keys():
        return False
    if graph[x][y] not in mapping[cur_val][tuple(dxdy)]:
        return False
               
    return True

#Breadth-First Search
def BFS_part1(graph, start):
    
    dist_graph = np.zeros(graph.shape, dtype=int)
    dist_graph[start[0],start[1]] = 9
    next_graph = np.empty([graph.shape[0]+2,graph.shape[1]+2], dtype=str)
    for i in range(0, len(next_graph)):
        for j in range(0, len(next_graph[i])):
            next_graph[i,j] = '.'
    next_graph[start[0]+1,start[1]+1] = 'S'
    
    dxdy = {'|': [[-1,0],[1,0]],
            '-': [[0,-1],[0,1]],
            'L': [[-1,0],[0,1]],
            'J': [[-1,0],[0,-1]],
            '7': [ [1,0],[0,-1]],
            'F': [ [1,0],[0,1]],
            'S': [[-1,0],[1,0],[0,-1],[0,1]]}
    
    queue = deque([start])
    dist = {start: 0}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        cur_val = graph[cur_pos[0]][cur_pos[1]]
        
        #print(cur_pos, cur_dist, cur_val)
        
        for xy in dxdy[cur_val]:
            nxt_dist = cur_dist + 1
            nxt_pos = (cur_pos[0]+xy[0], cur_pos[1]+xy[1])
            #print('\t', xy, graph[nxt_pos[0],nxt_pos[1]])
            
            if good_pos_part1(graph, nxt_pos, cur_val, xy) and nxt_pos not in dist.keys():
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
                dist_graph[nxt_pos[0],nxt_pos[1]] = nxt_dist
                next_graph[nxt_pos[0]+1,nxt_pos[1]+1] = graph[nxt_pos[0],nxt_pos[1]]
                
            elif nxt_pos in dist.keys() and dist[nxt_pos] == nxt_dist:
                #for line in dist_graph:
                #    print(line)
                #for line in next_graph:
                #    print(line)
                start = (start[0]+1, start[1]+1)
                
                #top
                if next_graph[start[0]-1, start[1]] == '|' or next_graph[start[0]-1, start[1]] == '7' or\
                   next_graph[start[0]-1, start[1]] == 'F':
                    #bottom
                    if next_graph[start[0]+1, start[1]] == '|' or\
                       next_graph[start[0]+1, start[1]] == 'J' or\
                       next_graph[start[0]+1, start[1]] == 'L':
                        next_graph[start[0], start[1]] = '|'
                    #right
                    elif next_graph[start[0], start[1]+1] == '-' or\
                         next_graph[start[0], start[1]+1] == 'J' or\
                         next_graph[start[0], start[1]+1] == '7':
                        next_graph[start[0], start[1]] = 'L'
                    #left
                    elif next_graph[start[0], start[1]-1] == '-' or\
                         next_graph[start[0], start[1]-1] == 'L' or\
                         next_graph[start[0], start[1]-1] == 'F':
                        next_graph[start[0], start[1]] = 'J'
                        
                #bottom
                elif next_graph[start[0]+1, start[1]] == '|' or next_graph[start[0]+1, start[1]] == 'L' or\
                   next_graph[start[0]+1, start[1]] == 'J':
                    #right
                    if next_graph[start[0], start[1]+1] == '-' or\
                       next_graph[start[0], start[1]+1] == 'J' or\
                       next_graph[start[0], start[1]+1] == '7':
                        next_graph[start[0], start[1]] = 'F'
                    #left
                    elif next_graph[start[0], start[1]-1] == '-' or\
                         next_graph[start[0], start[1]-1] == 'L' or\
                         next_graph[start[0], start[1]-1] == 'F':
                        next_graph[start[0], start[1]] = '7'
                        
                #left 
                elif next_graph[start[0], start[1]-1] == '-' or next_graph[start[0], start[1]-1] == 'L' or\
                   next_graph[start[0], start[1]-1] == 'F':
                    #right
                    if next_graph[start[0], start[1]+1] == '-' or\
                       next_graph[start[0], start[1]+1] == 'J' or\
                       next_graph[start[0], start[1]+1] == '7':
                        next_graph[start[0], start[1]] = '-'
                
                return nxt_dist, next_graph
    return -1

graph0, start0 = get_graph(test0)
dist0, ngraph0 = BFS_part1(graph0, start0)
print(dist0)
graph1, start1 = get_graph(test1)
dist1, ngraph1 = BFS_part1(graph1, start1)
print(dist1)
data_graph, data_start = get_graph(data)
data_dist, data_ngraph = BFS_part1(data_graph, data_start)
print('Part 1 result:', data_dist)

4
8
Part 1 result: 6846


In [4]:
test2 = ['...........',
         '.S-------7.',
         '.|F-----7|.',
         '.||.....||.',
         '.||.....||.',
         '.|L-7.F-J|.',
         '.|..|.|..|.',
         '.L--J.L--J.',
         '...........']

test3 = ['..........',
         '.S------7.',
         '.|F----7|.',
         '.||....||.',
         '.||....||.',
         '.|L-7F-J|.',
         '.|..||..|.',
         '.L--JL--J.',
         '..........']

test4 = ['.F----7F7F7F7F-7....',
         '.|F--7||||||||FJ....',
         '.||.FJ||||||||L7....',
         'FJL7L7LJLJ||LJ.L-7..',
         'L--J.L7...LJS7F-7L7.',
         '....F-J..F7FJ|L7L7L7',
         '....L7.F7||L7|.L7L7|',
         '.....|FJLJ|FJ|F7|.LJ',
         '....FJL-7.||.||||...',
         '....L---J.LJ.LJLJ...']

In [10]:
def get_os(xy):
    # 1
    #2X-2
    #-1
    if xy[0] == 1:
        os = 0
    elif xy[1] == 1:
        os = 2
    elif xy[1] == 2:
        os = -2
    elif xy[0] == -1:
        os = -1
    return os

def get_next_pos(graph, pos, dxdy):
    xy = deepcopy(dxdy)
    
    #also track outside
    os = get_os(xy)
    
    x = pos[0]+xy[0]
    y = pos[1]+xy[1]
    
    trcrt = False
    if pos == (5, 5):
        trcrt = True
        print('trcrt')
        print(pos, graph[pos[0],pos[1]])
        print((x,y), graph[x,y])
    
    while True:
        if x < 0 or x >= len(graph):
            return None
        if y < 0 or y >= len(graph[x]):
            return None

        if graph[x,y] == '.':
            if trcrt:
                print('trcrt', (x,y), '.')
            return (x,y)
        
        if graph[x,y] == '-' and (xy[0] == -1 or xy[0] == 1):
            return None
        if graph[x,y] == '|' and (xy[1] == -1 or xy[1] == 1):
            return None
    
        #JX  XL  
        #.F  7.

        #Diagonals
        if graph[x,y] == 'J' and xy[0] == -1 and graph[x+1,y+1] == 'F' and graph[x,y+1] == '.':
            if trcrt:
                print('trcrt', (x,y+1), graph[x,y])
            return (x,y+1)
        elif graph[x,y] == 'F' and xy[0] == 1 and graph[x-1,y-1] == 'J' and graph[x,y-1] == '.':
            if trcrt:
                print('trcrt', (x,y-1), graph[x,y])
            return (x,y-1)
        elif graph[x,y] == 'L' and xy[0] == -1 and graph[x+1,y-1] == '7' and graph[x,y-1] == '.':
            if trcrt:
                print('trcrt', (x,y-1), graph[x,y])
            return (x,y-1)
        elif graph[x,y] == '7' and xy[0] == 1 and graph[x-1,y+1] == 'L' and graph[x,y+1] == '.':
            if trcrt:
                print('trcrt', (x,y+1), graph[x,y])
            return (x,y+1)

        # 7F
        # ||
        # ||
        # JL

        if xy[0] == -1:
            #left side up
            if (graph[x,y] == 'J' or graph[x,y] == '|') and (graph[x,y+1] == 'L' or graph[x,y+1] == '|'):
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == '7' and (graph[x-1,y] == '-' or graph[x-1,y] == 'J'):
                xy[0] = 0
                xy[1] = -1
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == 'F' and graph[x,y+1] == '7':
                xy[0] *= -1
                x += xy[0]
                y += 1
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])

            #right side up
            elif (graph[x,y] == 'L' or graph[x,y] == '|') and (graph[x,y-1] == 'J' or graph[x,y-1] == '|'):
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == 'F' and (graph[x-1,y] == '-' or graph[x-1,y] == 'L'):
                xy[0] = 0
                xy[1] = 1
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == '7' and graph[x,y-1] == 'F':
                xy[0] *= -1
                x += xy[0]
                y -= 1
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
                
            elif (graph[x,y] == '7' or graph[x,y] == 'F') and\
                 (graph[x+1,y] == '|' or graph[x+1,y] == 'L' or graph[x+1,y] == 'J') and\
                 graph[x-1,y] == '.':
                #x += xy[0]
                if trcrt:
                    print('trcrt', (x-1,y), '.')
                return ((x-1,y))
                #print('trcrt', (x,y))
            else:
                return None

        elif xy[0] == 1:
            #left side down
            if (graph[x,y] == '7' or graph[x,y] == '|') and (graph[x,y+1] == 'F' or graph[x,y+1] == '|'):
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == 'J' and (graph[x+1,y] == '-' or graph[x+1,y] == '7'):
                xy[0] = 0
                xy[1] = -1
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == 'L' and graph[x,y+1] == 'J':
                xy[0] *= -1
                x += xy[0]
                y += 1
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y], xy)

            #right side down
            elif (graph[x,y] == 'F' or graph[x,y] == '|') and (graph[x,y-1] == '7' or graph[x,y-1] == '|'):
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == 'L' and (graph[x+1,y] == '-' or graph[x+1,y] == 'F'):
                xy[0] = 0
                xy[1] = 1
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == 'J' and graph[x,y-1] == 'L':
                xy[0] *= -1
                x += xy[0]
                y -= 1
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y], xy)
                
            elif (graph[x,y] == 'J' or graph[x,y] == 'L') and\
                 (graph[x-1,y] == '|' or graph[x-1,y] == '7' or graph[x-1,y] == 'F') and\
                graph[x+1,y] == '.':
                #x += xy[0]
                if trcrt:
                    print('trcrt', (x+1,y), '.')
                return ((x+1,y))
            else:
                return None

        # L--J
        # F--7
        elif xy[1] == -1:
            #top side left
            if (graph[x,y] == 'J' or graph[x,y] == '-') and (graph[x+1,y] == '7' or graph[x,y] == '-'):
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == 'L' and (graph[x,y-1] == '|' or graph[x,y-1] == 'J'):
                xy[0] = -1
                xy[1] = 0
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == 'F' and graph[x+1,y] == 'L':
                xy[1] *= -1
                x += 1
                y += xy[1]
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])

            #bottom side left
            elif (graph[x,y] == '7' or graph[x,y] == '-') and (graph[x+1,y] == 'J' or graph[x,y] == '-'):
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == 'F' and (graph[x,y-1] == '|' or graph[x,y-1] == '7'):
                xy[0] = 1
                xy[1] = 0
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == 'L' and graph[x-1,y] == 'F':
                xy[1] *= -1
                x -= 1
                y += xy[1]
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
                
            elif (graph[x,y] == 'L' or graph[x,y] == 'F') and\
                 (graph[x,y+1] == '-' or graph[x,y+1] == 'J' or graph[x,y+1] == '7') and\
                 graph[x,y-1] == '.':
                #y += xy[1]
                if trcrt:
                    print('trcrt', (x,y-1), '.')
                return (x,y-1)
            else:
                return None

        elif xy[1] == 1:
            #top side right
            if (graph[x,y] == 'L' or graph[x,y] == '-') and (graph[x+1,y] == 'F' or graph[x+1,y] == '-'):
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == 'J' and (graph[x,y+1] == '|' or graph[x,y+1] == 'L'):
                xy[0] = -1
                xy[1] = 0
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == '7' and graph[x+1,y] == 'J':
                xy[1] *= -1
                x += 1
                y += xy[1]
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])

            #bottom side right
            elif (graph[x,y] == 'F' or graph[x,y] == '-') and (graph[x+1,y] == 'L' or graph[x+1,y] == '-'):
                y += xy[1]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            elif graph[x,y] == '7' and (graph[x,y+1] == '|' or graph[x,y+1] == 'F'):
                xy[0] = 1
                xy[1] = 0
                x += xy[0]
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
            ##U turn
            elif graph[x,y] == 'J' and graph[x-1,y] == '7':
                xy[1] *= -1
                x -= 1
                y += xy[1]
                os *= -1
                if trcrt:
                    print('trcrt', (x,y), graph[x,y])
                
            elif (graph[x,y] == 'J' or graph[x,y] == '7') and\
                 (graph[x,y-1] == '-' or graph[x,y-1] == 'L' or graph[x,y-1] == 'F') and\
                 graph[x,y+1] == '.':
                #y += xy[1]
                if trcrt:
                    print('trcrt', (x,y+1), '.')
                return (x,y+1)
            else:
                return None

In [11]:
#Breadth-First Search
def BFS_part2(graph, start):
    
    INSIDE = [
              (5,8), (5,9), (5,10),
              (6,8), (6,9),
              (7,7)]
    
    dxdy = [[-1,0], [1,0], [0,-1], [0,1]]
    queue = deque([start])
    dist = {start: 0}
    
    outgraph = deepcopy(graph)
    outgraph[start[0],start[1]] = 'O'
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        cur_val = graph[cur_pos[0]][cur_pos[1]]
        
        print(cur_pos)
        for xy in dxdy:
            nxt_dist = cur_dist + 1
            nxt_pos = get_next_pos(graph, cur_pos, xy)
            
            if nxt_pos is not None and nxt_pos not in dist.keys():
                print('\t', nxt_pos)
                if nxt_pos in INSIDE:
                    print('\tHERE')
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
                outgraph[nxt_pos[0],nxt_pos[1]] = 'O'
        #print()
        
    graph_size = graph.shape[0]*graph.shape[1]
    path = np.where(graph != '.')
    path_size = len(path[0])
    outside = len(dist.keys())
    
    inside = graph_size - path_size - outside
    
    for i in range(1, len(outgraph)-1):
        print(''.join(outgraph[i][1:-1]))
    
    return inside

#print(BFS_part2(ngraph0, (0,0)))
#print()
#print(BFS_part2(ngraph1, (0,0)))
#print()
#graph2, start2 = get_graph(test2)
#dist2, ngraph2 = BFS_part1(graph2, start2)
#print(BFS_part2(ngraph2, (0,0)))
#print()
#graph3, start3 = get_graph(test3)
#dist3, ngraph3 = BFS_part1(graph3, start3)
#print(BFS_part2(ngraph3, (0,0)))
#print()
graph4, start4 = get_graph(test4)
dist4, ngraph4 = BFS_part1(graph4, start4)
print(BFS_part2(ngraph4, (0,0)))

(0, 0)
	 (1, 0)
	 (0, 1)
(1, 0)
	 (2, 0)
	 (1, 1)
(0, 1)
	 (0, 2)
(2, 0)
	 (3, 0)
	 (2, 1)
(1, 1)
(0, 2)
	 (0, 3)
(3, 0)
	 (4, 0)
	 (3, 1)
(2, 1)
(0, 3)
	 (0, 4)
(4, 0)
	 (5, 0)
(3, 1)
(0, 4)
	 (0, 5)
(5, 0)
	 (6, 0)
(0, 5)
	 (0, 6)
(6, 0)
	 (7, 0)
	 (6, 1)
(0, 6)
	 (0, 7)
(7, 0)
	 (8, 0)
	 (7, 1)
(6, 1)
	 (6, 2)
(0, 7)
	 (0, 14)
	 (0, 8)
(8, 0)
	 (9, 0)
	 (8, 1)
(7, 1)
	 (7, 2)
(6, 2)
	 (6, 3)
(0, 14)
	 (0, 13)
	 (0, 15)
(0, 8)
	 (0, 9)
(9, 0)
	 (10, 0)
	 (9, 1)
(8, 1)
	 (8, 2)
(7, 2)
	 (7, 3)
(6, 3)
	 (6, 4)
(0, 13)
	 (0, 12)
(0, 15)
	 (0, 16)
(0, 9)
	 (0, 10)
(10, 0)
	 (11, 0)
	 (10, 1)
(9, 1)
	 (9, 2)
(8, 2)
	 (8, 3)
(7, 3)
	 (7, 4)
(6, 4)
	 (5, 5)
(0, 12)
	 (0, 11)
(0, 16)
	 (0, 17)
(0, 10)
(11, 0)
	 (11, 1)
(10, 1)
	 (10, 2)
(9, 2)
	 (9, 3)
(8, 3)
	 (8, 4)
(7, 4)
(5, 5)
trcrt
(5, 5) .
(4, 5) L
trcrt
(5, 5) .
(6, 5) F
trcrt (6, 4) F
trcrt
(5, 5) .
(5, 4) J
trcrt
(5, 5) .
(5, 6) L
trcrt (5, 7) 7
trcrt (6, 6) -
trcrt (6, 5) F
trcrt (7, 6) 7
trcrt (7, 7) .
	 (7, 7)
	HERE
(0, 11)
(0, 